In [1]:
%load_ext autoreload
%autoreload 2

# Start

In [2]:
import random

from utils import logger
from game import WerewolfGame
from config import GAME_CONFIG, RoleType

# WerewolfGame

In [7]:
logger.info("Starting Werewolf Game Simulation...")
logger

2026-02-17 10:08:24,029 - WerewolfGame - INFO - Starting Werewolf Game Simulation...


<Logger WerewolfGame (DEBUG)>

In [8]:
game = WerewolfGame()

2026-02-17 10:08:24,219 - WerewolfGame - INFO - Game Initialized with roles:
2026-02-17 10:08:24,219 - WerewolfGame - INFO - Player 1 (狼人) - Alive 
2026-02-17 10:08:24,219 - WerewolfGame - INFO - Player 2 (狼人) - Alive 
2026-02-17 10:08:24,219 - WerewolfGame - INFO - Player 3 (狼人) - Alive 
2026-02-17 10:08:24,220 - WerewolfGame - INFO - Player 4 (狼人) - Alive 
2026-02-17 10:08:24,220 - WerewolfGame - INFO - Player 5 (平民) - Alive 
2026-02-17 10:08:24,220 - WerewolfGame - INFO - Player 6 (平民) - Alive 
2026-02-17 10:08:24,220 - WerewolfGame - INFO - Player 7 (平民) - Alive 
2026-02-17 10:08:24,221 - WerewolfGame - INFO - Player 8 (平民) - Alive 
2026-02-17 10:08:24,221 - WerewolfGame - INFO - Player 9 (预言家) - Alive 
2026-02-17 10:08:24,221 - WerewolfGame - INFO - Player 10 (女巫) - Alive 
2026-02-17 10:08:24,221 - WerewolfGame - INFO - Player 11 (猎人) - Alive 
2026-02-17 10:08:24,221 - WerewolfGame - INFO - Player 12 (白痴) - Alive 


In [15]:
game.players[0].knowledge_prob

{2: {<RoleType.WEREWOLF: '狼人'>: 1.0},
 3: {<RoleType.WEREWOLF: '狼人'>: 1.0},
 4: {<RoleType.WEREWOLF: '狼人'>: 1.0},
 5: {<RoleType.VILLAGER: '平民'>: 0.5,
  <RoleType.SEER: '预言家'>: 0.125,
  <RoleType.WITCH: '女巫'>: 0.125,
  <RoleType.HUNTER: '猎人'>: 0.125,
  <RoleType.IDIOT: '白痴'>: 0.125},
 6: {<RoleType.VILLAGER: '平民'>: 0.5,
  <RoleType.SEER: '预言家'>: 0.125,
  <RoleType.WITCH: '女巫'>: 0.125,
  <RoleType.HUNTER: '猎人'>: 0.125,
  <RoleType.IDIOT: '白痴'>: 0.125},
 7: {<RoleType.VILLAGER: '平民'>: 0.5,
  <RoleType.SEER: '预言家'>: 0.125,
  <RoleType.WITCH: '女巫'>: 0.125,
  <RoleType.HUNTER: '猎人'>: 0.125,
  <RoleType.IDIOT: '白痴'>: 0.125},
 8: {<RoleType.VILLAGER: '平民'>: 0.5,
  <RoleType.SEER: '预言家'>: 0.125,
  <RoleType.WITCH: '女巫'>: 0.125,
  <RoleType.HUNTER: '猎人'>: 0.125,
  <RoleType.IDIOT: '白痴'>: 0.125},
 9: {<RoleType.VILLAGER: '平民'>: 0.5,
  <RoleType.SEER: '预言家'>: 0.125,
  <RoleType.WITCH: '女巫'>: 0.125,
  <RoleType.HUNTER: '猎人'>: 0.125,
  <RoleType.IDIOT: '白痴'>: 0.125},
 10: {<RoleType.VILLAGER: '平民'>

## run_night

In [3]:
game = WerewolfGame()
deaths = {} 

2026-02-17 12:21:23,162 - WerewolfGame - INFO - Game Initialized with roles:
2026-02-17 12:21:23,163 - WerewolfGame - INFO - Player 1 (狼人) - Alive 
2026-02-17 12:21:23,163 - WerewolfGame - INFO - Player 2 (狼人) - Alive 
2026-02-17 12:21:23,163 - WerewolfGame - INFO - Player 3 (狼人) - Alive 
2026-02-17 12:21:23,163 - WerewolfGame - INFO - Player 4 (狼人) - Alive 
2026-02-17 12:21:23,163 - WerewolfGame - INFO - Player 5 (平民) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 6 (平民) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 7 (平民) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 8 (平民) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 9 (预言家) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 10 (女巫) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 11 (猎人) - Alive 
2026-02-17 12:21:23,164 - WerewolfGame - INFO - Player 12 (白痴) - Alive 


### Werewolf

In [4]:
wolves = game.get_players_by_role(RoleType.WEREWOLF)
alive_wolves = [p for p in wolves if p.is_alive]
wolf = alive_wolves[0]
print(wolf)

Player 1 (狼人) - Alive 


In [5]:
votes = {}
alive_players = game.get_alive_players()
for wolf in alive_wolves:
    # Delegate to Role
    target = wolf.role.choose_kill_target(alive_players, wolf.knowledge_prob)
    if target:
        votes[target] = votes.get(target, 0) + 1
votes

{<player.Player at 0x10ac34860>: 4}

In [6]:
max_votes = max(votes.values())
top_targets = [t for t, count in votes.items() if count == max_votes]
wolf_kill = random.choice(top_targets)
deaths[wolf_kill] = "Wolf"
f"Wolves decided to kill {target} (Votes: {max_votes}/{len(alive_wolves)})"

'Wolves decided to kill Player 5 (平民) - Alive  (Votes: 4/4)'

### Seer

In [7]:
seer_list = game.get_players_by_role(RoleType.SEER)
seer = seer_list[0] if seer_list else None
check_target = seer.role.choose_check_target(game.get_alive_players(), seer.knowledge_prob)
print(check_target)

Player 1 (狼人) - Alive 


In [8]:
seer.role.checked_players.append(check_target.id)
is_wolf = check_target.role.role_type == RoleType.WEREWOLF
fate = RoleType.WEREWOLF if is_wolf else RoleType.VILLAGER
seer.mark_role_certain(check_target.id, fate)
seer.knowledge_prob[check_target.id]

{<RoleType.WEREWOLF: '狼人'>: 1.0}

### Witch

In [9]:
witch_list = game.get_players_by_role(RoleType.WITCH)
witch = witch_list[0] if witch_list else None

In [10]:
witch.role.choose_save_decision(wolf_kill, witch.knowledge_prob)

True

In [11]:
witch.role.use_antidote()
del deaths[wolf_kill]
wolf_kill.saved = True

In [12]:
poison_target = witch.role.choose_poison_target([p for p in game.get_alive_players() if p != witch], witch.knowledge_prob)
poison_target.id

1

## run_day

In [13]:
game.day_count += 1 

### run_sheriff_election

In [14]:
game.run_sheriff_election()

2026-02-17 12:21:26,182 - WerewolfGame - INFO - Running Sheriff Election...
2026-02-17 12:21:26,182 - WerewolfGame - INFO - Player 9 (预言家) runs for Sheriff.
2026-02-17 12:21:26,182 - WerewolfGame - INFO - Player 10 (女巫) runs for Sheriff.
2026-02-17 12:21:26,183 - WerewolfGame - INFO - Candidates share information:
2026-02-17 12:21:26,183 - WerewolfGame - INFO - [Discussion] Seer 9 says: Player 1 is 狼人
2026-02-17 12:21:26,183 - WerewolfGame - INFO - [Discussion] Witch 10 says: I saved Player 5 last night! (Silver Water)
2026-02-17 12:21:26,183 - WerewolfGame - INFO - Sheriff is Player 9 (预言家)


In [16]:
game.players[6].knowledge_prob[1]

{<RoleType.WEREWOLF: '狼人'>: 1.0}

In [17]:
game.players[6].knowledge_prob[5]

{<RoleType.VILLAGER: '平民'>: 0.42857142857142855,
 <RoleType.SEER: '预言家'>: 0.14285714285714288,
 <RoleType.WITCH: '女巫'>: 0.14285714285714288,
 <RoleType.HUNTER: '猎人'>: 0.14285714285714288,
 <RoleType.IDIOT: '白痴'>: 0.14285714285714288}

### handle death

In [20]:
night_deaths = deaths
night_deaths

{}

### share_information

In [43]:
for p in game.get_alive_players():
    p.role.share_information(p, game.players)

2026-02-17 12:14:06,844 - WerewolfGame - INFO - [Discussion] Seer 9 says: Player 1 is 狼人
2026-02-17 12:14:06,844 - WerewolfGame - INFO - [Discussion] Witch 10 says: I saved Player 5 last night! (Silver Water)


In [47]:
game.players[6].knowledge_prob[5]

{<RoleType.VILLAGER: '平民'>: 0.42857142857142855,
 <RoleType.SEER: '预言家'>: 0.14285714285714288,
 <RoleType.WITCH: '女巫'>: 0.14285714285714288,
 <RoleType.HUNTER: '猎人'>: 0.14285714285714288,
 <RoleType.IDIOT: '白痴'>: 0.14285714285714288}

### run_sheriff_election

In [ ]:
candidates = game.get_alive_players()